In [25]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install --upgrade nbformat

You should consider upgrading via the 'c:\users\jaisw\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\jaisw\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\jaisw\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\jaisw\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


Pandas is an API

Pandas is actually set of software components , much of which is not even written in Python

In [26]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

In [27]:
#You create a dictionary, this is just data.
dict_={'a':[11,21,31],'b':[12,22,32]}

In [28]:
#When you create a Pandas object with the Dataframe constructor in API lingo, this is an "instance". 
#The data in the dictionary is passed along to the pandas API. You then use the dataframe to communicate with the API.

In [29]:
df=pd.DataFrame(dict_)
type(df)

pandas.core.frame.DataFrame

In [30]:
#When you call the method head the dataframe communicates with the API displaying the first few rows of the dataframe.

In [31]:
df.head()

,a,b
0,11,12
1,21,22
2,31,32


In [32]:
df.mean()

a    21.0
b    22.0
dtype: float64

Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin you want, vs_currency is the currency you want the price in, and days is how many days back from today you want.

In [33]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [34]:
type(bitcoin_data )

dict

In [35]:
bitcoin_data

{'prices': [[1672466519925, 16544.73713791309],
  [1672470122876, 16546.7345384431],
  [1672473719226, 16565.631689833943],
  [1672477200918, 16557.105440859435],
  [1672480930356, 16545.049525087477],
  [1672484426060, 16567.29622449143],
  [1672488089518, 16579.76890855689],
  [1672491636756, 16590.381641828037],
  [1672495380145, 16611.607871078406],
  [1672498902539, 16602.348708093585],
  [1672502477740, 16598.821535416228],
  [1672506117678, 16609.04624489281],
  [1672509692647, 16583.059697606008],
  [1672513318991, 16580.246393829664],
  [1672516925641, 16581.7838283338],
  [1672520467450, 16574.68823928953],
  [1672524120199, 16551.80537614912],
  [1672527704225, 16521.815520470438],
  [1672531295950, 16542.456148037014],
  [1672534905941, 16543.034201213337],
  [1672538480919, 16558.02684830619],
  [1672542115335, 16546.030320462633],
  [1672545719438, 16539.208141738363],
  [1672549210910, 16524.263454746153],
  [1672552945712, 16530.65757502096],
  [1672556576482, 16544.695

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.


In [36]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1672466519925, 16544.73713791309],
 [1672470122876, 16546.7345384431],
 [1672473719226, 16565.631689833943],
 [1672477200918, 16557.105440859435],
 [1672480930356, 16545.049525087477]]

finally lets turn this data into a Pandas DataFrame.

In [37]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

In [38]:
data

,TimeStamp,Price
0,1672466519925,16544.737138
1,1672470122876,16546.734538
2,1672473719226,16565.631690
3,1672477200918,16557.105441
4,1672480930356,16545.049525
...,...,...
718,1675044134373,23674.939283
719,1675047622415,23645.019636
720,1675051304615,23683.353712
721,1675054917930,23739.569049


Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [39]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

In [40]:
data

,TimeStamp,Price,date
0,1672466519925,16544.737138,2022-12-31
1,1672470122876,16546.734538,2022-12-31
2,1672473719226,16565.631690,2022-12-31
3,1672477200918,16557.105441,2022-12-31
4,1672480930356,16545.049525,2022-12-31
...,...,...,...
718,1675044134373,23674.939283,2023-01-30
719,1675047622415,23645.019636,2023-01-30
720,1675051304615,23683.353712,2023-01-30
721,1675054917930,23739.569049,2023-01-30


Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [41]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

In [42]:
candlestick_data

date         Price                                          
                         min           max         first          last
0   2022-12-31  16544.737138  16611.607871  16544.737138  16583.059698
1   2023-01-01  16518.854339  16587.610925  16580.246394  16587.610925
2   2023-01-02  16573.799637  16732.002143  16591.826140  16716.397662
3   2023-01-03  16625.153163  16750.982601  16718.037278  16625.153163
4   2023-01-04  16637.275962  16925.521898  16644.549378  16925.521898
5   2023-01-05  16792.659516  16972.972810  16972.972810  16825.480381
6   2023-01-06  16726.147662  16863.195805  16840.170936  16817.258045
7   2023-01-07  16843.073698  16973.698807  16843.073698  16939.212783
8   2023-01-08  16931.037714  16974.138894  16940.428941  16935.442111
9   2023-01-09  16932.986571  17359.371371  16932.986571  17359.371371
10  2023-01-10  17184.023314  17354.864476  17354.864476  17329.023067
11  2023-01-11  17343.596846  17489.899095  17428.689900  17390.595694
12  2023-01-12  17476.451701  18794.607164  17476.451701  18794.607164
13  2023-01-13  18792.320558  19290.275055  18830.733127  19290.275055
14  2023-01-14  19344.381316  21083.034367  19344.546882  20743.696202
15  2023-01-15  20606.546176  20956.258670  20754.500110  20889.403820
16  2023-01-16  20700.805745  21252.809286  20700.805745  21100.504619
17  2023-01-17  21035.724169  21339.084565  21218.833093  21198.319122
18  2023-01-18  20532.907208  21470.684643  21356.441022  20934.375794
19  2023-01-19  20694.121613  20937.393961  20937.393961  20852.259374
20  2023-01-20  20935.470397  21353.082784  20960.404359  21353.082784
21  2023-01-21  21392.550652  23259.491423  21392.550652  23259.491423
22  2023-01-22  22730.688504  23282.403316  23282.403316  22922.036774
23  2023-01-23  22539.169943  22970.847176  22851.486498  22970.847176
24  2023-01-24  22725.853464  23137.668585  23066.963598  22903.912623
25  2023-01-25  22437.676767  23021.872378  23021.872378  22581.523269
26  2023-01-26  22702.124188  23399.023333  22702.124188  23007.269511
27  2023-01-27  22651.832046  23284.281217  23048.701384  23284.281217
28  2023-01-28  22954.372104  23232.400809  23205.749284  23048.001140
29  2023-01-29  23002.994002  23611.041600  23015.997448  23605.333603
30  2023-01-30  23645.019636  23907.560620  23696.748061  23722.502784

Finally we are now ready to use plotly to create our Candlestick Chart.

In [43]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()